In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_rumours.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0


In [6]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [7]:
sbert_embeddings.shape

torch.Size([5568, 384])

# Baseline: FFN using signatures

history_length = 20

In [8]:
dim_reduce_methods = ["umap"]
num_epochs = 100
log_sig = True
log_signature_dimensions_and_sig_depths = [ (10, 3)]#[(28, 2), (10, 3), (6, 4)]
hidden_dim_sizes = [[32,32]]#[[32,32], [64,64], [128,128], [256,256]]
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
learning_rates = [1e-4]#[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [9]:
history_length = 20
ffn_logsignature_umap_20, best_ffn_logsignature_umap_20, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=log_sig,
    dim_reduce_methods=dim_reduce_methods,
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths, 
    history_lengths=[history_length],
    path_indices=None,
    data_split_seed=123,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]


[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.














































































































































































































































































































100%|██████████| 5568/5568 [00:17<00:00, 321.54it/s]













100%|██████████| 1/1 [01:05<00:00, 65.06s/it]



100%|██████████| 1/1 [01:05<00:00, 65.06s/it]


100%|██████████| 1/1 [01:05<00:00, 65.06s/it]

100%|██████████| 1/1 [03:05<00:00, 185.26s/it]


[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [00:16<00:00, 334.82it/s]


In [11]:
best_ffn_logsignature_umap_20["f1"].mean()

0.5627569541197252

In [12]:
np.stack(best_ffn_logsignature_umap_20["f1_scores"]).mean(axis=0)

array([0.70831814, 0.41719577])